The goal of this script is to scrape information on the top20 rated movies on imdb, then adjust its ratings based on number of ratings given and number of Oscars won. The script could be used to expand it to the top 250 movies. The first part of the script scrapes the necessary information: title of movie, imdb rating, number of ratings and number of oscars won. The second part of the script creates the adjusted ratings given the information scraped and criteria.

In [4]:
import requests
existing_url = 'https://www.imdb.com/chart/top/'
response = requests.get(existing_url)
print(response.status_code)

200


In [30]:
from bs4 import BeautifulSoup
response = requests.get(existing_url)
document = response.content
soup = BeautifulSoup(document, 'html.parser')
##print(soup.prettify())

Checking the site and the html code, the title of movie, rating and number of ratings appear in the top250 page. THe number of Oscars won does not appear but there are links to the separate movie pages where we can get the Oscars information from. Start with scraping these links for the site.

In [31]:
## soup.find_all('a') ## finding all the links

In [7]:
titlelist=[]
for url in soup.find_all('a'):
        titlelist.append(url.get('href'))
titlelist2=[]
substring = "title/tt" ## with inspecting the site we can find that this is the structure how links to separate pages are built.
for item in titlelist:
    try:
        item.index(substring)
    except ValueError: ## error handling, if it doesn't find the string, we can ignore the error
        pass
    except AttributeError: ## error handling, if it doesn't find the string, we can ignore the error
        pass
    else:
        titlelist2.append(item)
## checking the titlelist2 list, we can find that all movie page links appeared twice, let's get rid of the extra ones, but
## have to be careful to keep the order (so a dictionary won't work)
titlelist_url = []
for i in titlelist2:
    if i not in titlelist_url:
        titlelist_url.append(i)
titlelist_url ## list that contains all the separate movie links

['/title/tt0111161/',
 '/title/tt0068646/',
 '/title/tt0468569/',
 '/title/tt0071562/',
 '/title/tt0050083/',
 '/title/tt0108052/',
 '/title/tt0167260/',
 '/title/tt0110912/',
 '/title/tt0120737/',
 '/title/tt0060196/',
 '/title/tt0109830/',
 '/title/tt0137523/',
 '/title/tt1375666/',
 '/title/tt0167261/',
 '/title/tt0080684/',
 '/title/tt0133093/',
 '/title/tt0099685/',
 '/title/tt0073486/',
 '/title/tt0114369/',
 '/title/tt0047478/',
 '/title/tt0038650/',
 '/title/tt0102926/',
 '/title/tt0120815/',
 '/title/tt0317248/',
 '/title/tt0118799/',
 '/title/tt0120689/',
 '/title/tt0076759/',
 '/title/tt0816692/',
 '/title/tt0103064/',
 '/title/tt0088763/',
 '/title/tt0245429/',
 '/title/tt0054215/',
 '/title/tt0110413/',
 '/title/tt0253474/',
 '/title/tt6751668/',
 '/title/tt0110357/',
 '/title/tt0172495/',
 '/title/tt0120586/',
 '/title/tt0114814/',
 '/title/tt0407887/',
 '/title/tt0482571/',
 '/title/tt0034583/',
 '/title/tt2582802/',
 '/title/tt1675434/',
 '/title/tt0027977/',
 '/title/t

We have the list of links, let's take that away for now and scrape the rest of the information, starting with the titles.

In [32]:
titles=soup.find_all('td', class_="titleColumn") ##inspecting the html, I found the class titleColumn to have the names
## titles

In [9]:
top20_titles=[]
for i in range(20): ## only need the top 20 names
    temptitle=titles[i].find_all('a')[0]
    start = str(temptitle).find(">") + len(">") ## checking the previous list, we can find between which strings the title appears
    end = str(temptitle).find("</a>")
    titlename = str(temptitle)[start:end] ## has to be string datatype
    top20_titles.append(titlename)
top20_titles ## list containing the top 20 titles in order

['A remény rabjai',
 'A keresztapa',
 'A sötét lovag',
 'A keresztapa II',
 'Tizenkét dühös ember',
 'Schindler listája',
 'A Gyűrűk Ura: A király visszatér',
 'Ponyvaregény',
 'A Gyűrűk Ura: A gyűrű szövetsége',
 'A Jó, a Rossz és a Csúf',
 'Forrest Gump',
 'Harcosok klubja',
 'Eredet',
 'A Gyűrűk Ura: A két torony',
 'A Birodalom visszavág',
 'Mátrix',
 'Nagymenők',
 'Száll a kakukk fészkére',
 'Hetedik',
 'A hét szamuráj']

In [33]:
ratingslist=[]
ratingslist=soup.find_all('td', class_="ratingColumn") ## the ratingColumn class has the rating and the number of ratings information
## ratingslist

In [11]:
top20_ratings=[]
for i in range(20):
    temptitle=ratingslist[2*i].find_all('strong')[0]
    start = str(temptitle).find('title="') + len('title="') ## similar to before just need to find the string structure
    end = str(temptitle).find(" based")
    rankname = str(temptitle)[start:end]
    top20_ratings.append(float(rankname)) ## has to be float datatype
top20_ratings  ## list containing the top20 ratings in order

[9.2,
 9.2,
 9.0,
 9.0,
 9.0,
 8.9,
 8.9,
 8.9,
 8.8,
 8.8,
 8.8,
 8.8,
 8.7,
 8.7,
 8.7,
 8.7,
 8.7,
 8.6,
 8.6,
 8.6]

In [12]:
top20_ratingnum=[]
for i in range(20):
    temptitle=ratingslist[2*i].find_all('strong')[0]
    start = str(temptitle).find('on ') + len('on ') ## similar to before just need to find the string structure
    end = str(temptitle).find(" user")
    rankname = str(temptitle)[start:end]
    rankname = rankname.replace(",","") ## the numbers appear in an x,yyy,zzz format, removing the ,-s to be able to handle as integer
    top20_ratingnum.append(int(rankname)) ## has to be integer datatype
top20_ratingnum ## list containing the top 20 movies' number of ratings in correct order

[2567457,
 1767424,
 2531677,
 1222909,
 758254,
 1308267,
 1766463,
 1970993,
 1788204,
 738538,
 1980359,
 2020249,
 2253338,
 1595891,
 1242298,
 1848097,
 1107933,
 978212,
 1574210,
 337317]

For the number of oscars we are going back to the separate movie pages list. We loop through this list, going separately to the top 20 movies' pages and scraping the oscar information there. I found 3 separate structures for listing awards: 1. "Won x Oscars", 2. "Nominated for x Oscars" 3. "Won x awards" (that are not oscars). Only in the first one the movie did actuallz win any oscars, there we need the X, otherwise we just need to add 0.

In [ ]:
## this runs a little slow (~30 seconds)

start_url = 'https://www.imdb.com' ##first part of the url is always this
oscarlist=[]
substring_oscar="Oscar" ##the string we are looking for during the scraping
for end_url in titlelist_url[0:20]: ##the second part of the url is coming from the prev created url list
    url=start_url+end_url
    response2=requests.get(url)
    document2 = response2.content
    soupurl = BeautifulSoup(document2, 'html.parser')
    oscarsouplist=list(soupurl.find_all('a', class_="ipc-metadata-list-item__label ipc-metadata-list-item__label--link"))
    ## I found this long class to search for during inspecting.
    
    tempchecker=0
    ## we are going through all the strings within the class, if anywhere Oscar string appears, we can actually find the
    ## awards/nominations, else we just add No Oscar to the oscarlist list
    for item2 in oscarsouplist:
        if substring_oscar in str(item2): 
            tempchecker=1
    if tempchecker == 0:
        oscarlist.append("No Oscar")
    else:
        for item in oscarsouplist:
            try:
                str(item).index(substring_oscar)
            except ValueError:
                pass
            except AttributeError:
                pass
            else:
                oscarlist.append(str(item))
## oscarlist

In [ ]:
len(oscarlist) ##expecting 20 items, checking to make sure there wasn't anything in the data structure I didn't notice

In [19]:
##We only need the number if "Won" appears in the itemm, otherwise the movie won 0 Oscars.

number_of_oscars = []
won_oscars = "Won"
for item in oscarlist:
    if won_oscars in item:
        start = str(item).find("Won ") + len("Won ")
        end = str(item).find(" Oscar")
        oscar_num = str(item)[start:end]
        number_of_oscars.append(int(oscar_num)) ## has to be integer
    else:
        number_of_oscars.append(0)
number_of_oscars ## list containing number of oscars won

[0, 3, 2, 6, 0, 7, 11, 1, 4, 0, 6, 0, 4, 2, 1, 4, 1, 5, 0, 0]

In [20]:
## Creating an original rating list to the table

ranks=[]
for i in range(20):
    ranks.append(i+1)
ranks

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

The scraper part is ready with this, we have 4 lists (20 items each) with Title of movie, original rating, number of ratings and number of oscars won along with an original rank. I am creating a pandas dataframe from these.

In [21]:
import pandas as pd
df = pd.DataFrame(list(zip(ranks, top20_titles,top20_ratings,top20_ratingnum,number_of_oscars)),
               columns =['Original Rank', 'Title','Original Rating','Number of Ratings','Oscars Won'])
df

,Original Rank,Title,Original Rating,Number of Ratings,Oscars Won
0,1,A remény rabjai,9.2,2567457,0
1,2,A keresztapa,9.2,1767424,3
2,3,A sötét lovag,9.0,2531677,2
3,4,A keresztapa II,9.0,1222909,6
4,5,Tizenkét dühös ember,9.0,758254,0
5,6,Schindler listája,8.9,1308267,7
6,7,A Gyűrűk Ura: A király visszatér,8.9,1766463,11
7,8,Ponyvaregény,8.9,1970993,1
8,9,A Gyűrűk Ura: A gyűrű szövetsége,8.8,1788204,4
9,10,"A Jó, a Rossz és a Csúf",8.8,738538,0


In [22]:
import numpy as np

Given the below instructions how the review penalizer works:
Find the film with the maximum number of reviews (remember, out of the TOP 20 only). This is going to be the benchmark. Compare every movie’s number of reviews to this and penalize each of them based on the following rule: Every 100k deviation from the maximum translates to a point deduction of 0.1. 

In [23]:
max_ratings=np.max(df.get("Number of Ratings")) ## find the maximum number of ratings
ratings_adjustment=df.get("Number of Ratings")-max_ratings ## let's subtract the maximum from each item                           
ratings_adjustment = ratings_adjustment.tolist()
ratings_adjustment_penalty = [int(item / 100000)/10 for item in ratings_adjustment]

## explanation: int(item / 100000) gets every 100k deviation from the maximum (but it does not round up, so e.g. if an item is
## 199,000 less than the maximum, it still only gets penalized 0.1) The /10 is needed the point deduction to be 0.1

ratings_adjustment_penalty ## the list how much we need to penalize each movie (in order)

[0.0,
 -0.8,
 0.0,
 -1.3,
 -1.8,
 -1.2,
 -0.8,
 -0.5,
 -0.7,
 -1.8,
 -0.5,
 -0.5,
 -0.3,
 -0.9,
 -1.3,
 -0.7,
 -1.4,
 -1.5,
 -0.9,
 -2.2]

Given the below instructions how the won oscars adjustment works:

 1 or 2 oscars → 0.3 point,
 3 or 5 oscars → 0.5 point,
 6 - 10 oscars → 1 point,
 10+ oscars → 1.5 point

In [24]:
## looping through and creating a simple if statement solves this

oscar_adjustments=[]
for item in number_of_oscars:
    if item<1:
        oscar_adjustments.append(0)
    elif item<3:
        oscar_adjustments.append(0.3)
    elif item<6:
        oscar_adjustments.append(0.5)
    elif item<11:
        oscar_adjustments.append(1)
    else:
        oscar_adjustments.append(1.5)
oscar_adjustments  ## the list with the adjusted ratings for won oscars

[0,
 0.5,
 0.3,
 1,
 0,
 1,
 1.5,
 0.3,
 0.5,
 0,
 1,
 0,
 0.5,
 0.3,
 0.3,
 0.5,
 0.3,
 0.5,
 0,
 0]

In [25]:
## simply adding the two adjusted ratings to the original ratings.

adjusted_rating = [x + y + z for (x, y , z) in zip(top20_ratings, ratings_adjustment_penalty,oscar_adjustments)]
adjusted_rating

[9.2,
 8.899999999999999,
 9.3,
 8.7,
 7.2,
 8.7,
 9.6,
 8.700000000000001,
 8.600000000000001,
 7.000000000000001,
 9.3,
 8.3,
 8.899999999999999,
 8.1,
 7.699999999999999,
 8.5,
 7.599999999999999,
 7.6,
 7.699999999999999,
 6.3999999999999995]

In [26]:
## computation messes up some decimals, let's round them back

adjusted_rating = [round(num, 1) for num in adjusted_rating]
adjusted_rating

[9.2,
 8.9,
 9.3,
 8.7,
 7.2,
 8.7,
 9.6,
 8.7,
 8.6,
 7.0,
 9.3,
 8.3,
 8.9,
 8.1,
 7.7,
 8.5,
 7.6,
 7.6,
 7.7,
 6.4]

In [27]:
## Add the newly calculated information to the dataframe.

df['Ratings Number Penalty'] = ratings_adjustment_penalty
df['Oscars Adjustment'] = oscar_adjustments
df['Adjusted Rating'] = adjusted_rating
df

,Original Rank,Title,Original Rating,Number of Ratings,Oscars Won,Ratings Number Penalty,Oscars Adjustment,Adjusted Rating
0,1,A remény rabjai,9.2,2567457,0,0.0,0.0,9.2
1,2,A keresztapa,9.2,1767424,3,-0.8,0.5,8.9
2,3,A sötét lovag,9.0,2531677,2,0.0,0.3,9.3
3,4,A keresztapa II,9.0,1222909,6,-1.3,1.0,8.7
4,5,Tizenkét dühös ember,9.0,758254,0,-1.8,0.0,7.2
5,6,Schindler listája,8.9,1308267,7,-1.2,1.0,8.7
6,7,A Gyűrűk Ura: A király visszatér,8.9,1766463,11,-0.8,1.5,9.6
7,8,Ponyvaregény,8.9,1970993,1,-0.5,0.3,8.7
8,9,A Gyűrűk Ura: A gyűrű szövetsége,8.8,1788204,4,-0.7,0.5,8.6
9,10,"A Jó, a Rossz és a Csúf",8.8,738538,0,-1.8,0.0,7.0


In [29]:
## Creating a csv given this was a requirement as well

df.to_csv('imdb_top20_adjusted_rating.csv',index=False)